![Ejemplo 6](images/Ej6.png)

In [219]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Leemos los datos de las corrientes calientes

In [220]:
Hot_datos = {0: [0,0], 1: [600,0], 2: [150,75], 3: [1650,825],4: [900,450], 5:[0,450]}
Hot_df = pd.DataFrame(Hot_datos,index=['H1','H2'])
Hot_df

,0,1,2,3,4,5
H1,0,600,150,1650,900,0
H2,0,0,75,825,450,450


E igualmente de las frías.

In [221]:
Cold_datos =  {0: [0,0], 1:[0,0], 2: [0,200], 3: [1100,2200], 4: [600,0], 5:[600,0]}
Cold_df = pd.DataFrame(Cold_datos,index=['C1','C2'])
Cold_df

,0,1,2,3,4,5
C1,0,0,0,1100,600,600
C2,0,0,200,2200,0,0


In [222]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
s=hotst.copy()
s.insert(0,'S')
w=coldst.copy()
w.insert(0,'W')

model.i=Set(initialize=hotst)
model.j=Set(initialize=coldst)
model.s=Set(initialize=s)
model.S=Set(initialize='S')
model.w=Set(initialize=w)
model.W=Set(initialize='W')
model.k=Set(initialize=Hot_df.columns)


Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [223]:
model.pprint()

7 Set Declarations
    S : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :  {'S',}
    W : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :  {'W',}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'H1', 'H2'}
    j : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'C1', 'C2'}
    k : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    s : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'S', 'H1', 'H2'}
    w : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :   

Creamos ahora las variables del modelo. Hay i potenciales aportes de calor por utilities calientes (Qs), uno en cada intervalo, al igual que de frías (Qw). De esta forma damos la oportunidad de que entren las utilities en cualquier nivel de temperatura. Creamos también i residuos (R), aunque sabemos que no habrá residuo del intervalo 0 ni tampoco del intervalo 9, lo cual implementaremos como dos restricciones diferentes.

In [224]:
Qs = model.Qs = Var(model.k,within = NonNegativeReals) 
Qw = model.Qw = Var(model.k,within = NonNegativeReals) 
R = model.R = Var(model.s,model.k,within = NonNegativeReals)
Q = model.Q = Var(model.s,model.w,model.k,within = NonNegativeReals)

Buscamos la minimización del vapor del intervalo 1 más la utility fría del intervalo 9.

In [225]:
model.util = Objective(expr = model.Qs[1] + model.Qw[5])

Constraints

In [226]:
nk = list(model.k)[1:] 
model.C1 = ConstraintList()
for k in nk:
    for i in model.i:
        model.C1.add(
        R[i,k-1]+Hot_df.loc[i,k] == R[i,k]+sum(Q[i,j,k] for j in model.j)+sum(Q[i,w,k] for w in model.W)
        )

model.C2 = ConstraintList()
for k in nk:
    for s in model.S:
        model.C2.add(
        R[s,k-1]+Qs[k] == R[s,k]+sum(Q[s,j,k] for j in model.j)
        )        

model.C3 = ConstraintList()
for k in nk:
    for j in model.j:
        model.C3.add(
        sum(Q[i,j,k] for i in model.i)+ sum(Q[i,j,k] for i in model.S) == Cold_df.loc[j,k]
        )

model.C4 = ConstraintList()
for k in nk:
        model.C4.add(
        sum(Q[i,'W',k] for i in model.i)  == Qw[k]
        )


model.C5 = ConstraintList()
for s in model.s:
    model.C5.add(
        R[s,0]==0
        )

model.C6 = ConstraintList()
for s in model.s:
    model.C6.add(
        R[s,5]==0
        )

nm = list(model.k)[0:1]+list(model.k)[2:]
model.C7 = ConstraintList()
for m in nm:
    model.C7.add(
        Qs[m]==0
        )

nn = list(model.k)[0:5]
model.C8 = ConstraintList()
for n in nn:
    model.C8.add(
        Qw[n]==0
        )

model.C9 = ConstraintList()
for k in model.k:
    model.C9.add(
        Q['S','W',k] ==0
    )

Resolvemos el modelo

In [227]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

18 Set Declarations
    C1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C4_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C5_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    C6_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    C7_index : Size=1, Index=N

In [228]:
Qss = value(model.Qs[1])
Qww = value(model.Qw[5])
print('Cold utility = {0:2.2f}, Hot utility = {1:2.2f}'.format(Qww, Qss))

Cold utility = 600.00, Hot utility = 200.00


Ahora resolvemos el problema de mínimo número de intercambios

In [229]:
U = {'C1': [200, 2300,1800], 'C2': [200, 2400,1800], 'W':[0,600,600]}
U_df = pd.DataFrame(U, index=['S', 'H1', 'H2'])
U_df

,C1,C2,W
S,200,200,0
H1,2300,2400,600
H2,1800,1800,600


In [230]:
model.y=Var(model.s,model.w, within=Binary)

model.C10 = ConstraintList()
for s in model.s:
    for w in model.w:
        model.C10.add(
        sum(Q[s,w,k] for k in model.k)-U_df.loc[s,w]*model.y[s,w] <=0
        )    

In [231]:
model.util.deactivate()
model.sumy = Objective(expr=sum(model.y[s,w] for s in model.s for w in model.w))

In [232]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

20 Set Declarations
    C10_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    C1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C4_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C5_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    C6_inde

In [233]:
modelNLP = ConcreteModel()

In [234]:
fH1I_1 = modelNLP.fH1I_1 = Var(bounds=(0,30), initialize=10)
fH1I_2 = modelNLP.fH1I_2 = Var(bounds=(0,30), initialize=10)
fC1I_1 = modelNLP.fC1I_1 = Var(bounds=(0,20), initialize=10)
fC1I_2 = modelNLP.fC1I_2 = Var(bounds=(0,20), initialize=10)

fH1E_1 = modelNLP.fH1E_1 = Var(bounds=(0,30), initialize=10)
fH1E_2 = modelNLP.fH1E_2 = Var(bounds=(0,30), initialize=10)
fC1E_1 = modelNLP.fC1E_1 = Var(bounds=(0,20), initialize=10)
fC1E_2 = modelNLP.fC1E_2 = Var(bounds=(0,20), initialize=10)



TH1I_1 = modelNLP.TH1I_1 = Var(bounds=(293,443),initialize=442)
TH1I_2 = modelNLP.TH1I_2 = Var(bounds=(293,443),initialize=442)
TC1I_1 = modelNLP.TC1I_1 = Var(bounds=(293,443),initialize=298)
TC1I_2 = modelNLP.TC1I_2 = Var(bounds=(293,443),initialize=298)

TH1E = modelNLP.TH1E = Var(bounds=(293,443),initialize=410)
TH2E = modelNLP.TH2E = Var(bounds=(293,443),initialize=410)
TC1E = modelNLP.TC1E = Var(bounds=(293,443),initialize=310)
TC2E = modelNLP.TC2E = Var(bounds=(293,443),initialize=358)


TH1E_1 = modelNLP.TH1E_1 = Var(bounds=(293,443),initialize=398)
TH1E_2 = modelNLP.TH1E_2 = Var(bounds=(293,443),initialize=398)
TH1F = modelNLP.TH1F = Var(bounds=(293,443),initialize=380)

TC1E_1 = modelNLP.TC1E_1 = Var(bounds=(293,443),initialize=312)
TC1E_2 = modelNLP.TC1E_2 = Var(bounds=(293,443),initialize=315)
TC1F = modelNLP.TC1F = Var(bounds=(293,443),initialize=370)

DT1H1C1_1 = modelNLP.DT1H1C1_1 = Var(bounds=(10,500),initialize=10)
DT1H1C1_2 = modelNLP.DT1H1C1_2 = Var(bounds=(10,500),initialize=10)
DT2H1C1_1 = modelNLP.DT2H1C1_1 = Var(bounds=(10,500),initialize=10)
DT2H1C1_2 = modelNLP.DT2H1C1_2 = Var(bounds=(10,500),initialize=10)

DT1H1C2_1 = modelNLP.DT1H1C2_1 = Var(bounds=(10,500),initialize=10)
DT1H1C2_2 = modelNLP.DT1H1C2_2 = Var(bounds=(10,500),initialize=10)
DT2H1C2_1 = modelNLP.DT2H1C2_1 = Var(bounds=(10,500),initialize=10)
DT2H1C2_2 = modelNLP.DT2H1C2_2 = Var(bounds=(10,500),initialize=10)

DT1H2C1_1 = modelNLP.DT1H2C1_1 = Var(bounds=(10,500),initialize=10)
DT1H2C1_2 = modelNLP.DT1H2C1_2 = Var(bounds=(10,500),initialize=10)
DT2H2C1_1 = modelNLP.DT2H2C1_1 = Var(bounds=(10,500),initialize=10)
DT2H2C1_2 = modelNLP.DT2H2C1_2 = Var(bounds=(10,500),initialize=10)

QH1C1_1 = modelNLP.QH1C1_1 = Var(bounds=(0.000001,300), initialize = 100)
QH1C1_2 = modelNLP.QH1C1_2 = Var(bounds=(0.000001,300), initialize = 100)
QH1C2_1 = modelNLP.QH1C2_1 = Var(bounds=(0.000001,2400), initialize = 100)
QH1C2_2 = modelNLP.QH1C2_2 = Var(bounds=(0.000001,2400), initialize = 100)
QH2C1_1 = modelNLP.QH2C1_1 = Var(bounds=(0.000001,1800), initialize = 100)
QH2C1_2 = modelNLP.QH2C1_2 = Var(bounds=(0.000001,1800), initialize = 100)

LMTD11_1 =modelNLP.LMTD11_1 = Var(bounds=(0.0000,1e6), initialize = 10)
LMTD11_2 =modelNLP.LMTD11_2 = Var(bounds=(0.0000,1e6), initialize = 10)
LMTD12_1 =modelNLP.LMTD12_1 = Var(bounds=(0.0000,1e6), initialize = 10)
LMTD12_2 =modelNLP.LMTD12_2 = Var(bounds=(0.0000,1e6), initialize = 10)
LMTD21_1 =modelNLP.LMTD21_1 = Var(bounds=(0.0000,1e6), initialize = 10)
LMTD21_2 =modelNLP.LMTD21_2 = Var(bounds=(0.0000,1e6), initialize = 10)


In [235]:
modelNLP.SPLIT1 = Constraint(expr=fC1I_1+fC1I_2==20)
modelNLP.SPLIT2 = Constraint(expr=fC1E_1+fC1E_2==20)
modelNLP.SPLIT3 = Constraint(expr=fH1I_1+fH1I_2==30)
modelNLP.SPLIT4 = Constraint(expr=fH1E_1+fH1E_2==30)

modelNLP.EN_MIX1 = Constraint(expr=TC1I_1*fC1I_1+TC1I_2*fC1I_2==20*TC1E)
modelNLP.EN_MIX2 = Constraint(expr=TC1E_1*fC1E_1+TC1E_2*fC1E_2==20*TC1F)
modelNLP.EN_MIX3 = Constraint(expr=TH1I_1*fH1I_1+TH1I_2*fH1I_2==30*TH1E)
modelNLP.EN_MIX4 = Constraint(expr=TH1E_1*fH1E_1+TH1E_2*fH1E_2==30*TH1F)

modelNLP.HEATEXH1C1_1 = Constraint(expr=fC1E_1*(TC1E_1-TC1E)==fH1I_1*(443-TH1I_1))
modelNLP.HEATEXH1C1_2 = Constraint(expr=fC1I_1*(TC1I_1-293)==fH1E_1*(TH1E-TH1E_1))
modelNLP.HEATEXH1C2_1 = Constraint(expr=40*(413-TC2E)==fH1I_2*(443-TH1I_2))
modelNLP.HEATEXH1C2_2 = Constraint(expr=40*(TC2E-353)==fH1E_2*(TH1E-TH1E_2))
modelNLP.HEATEXH2C1_1 = Constraint(expr=15*(423-TH2E)==fC1E_2*(TC1E_2-TC2E))
modelNLP.HEATEXH2C1_2 = Constraint(expr=15*(TH2E-303)==fC1I_2*(TC1I_2-293))

modelNLP.sumQH1C1 = Constraint(expr=fC1E_1*(TC1E_1-TC1E)+fC1I_1*(TC1I_1-293)==300)
modelNLP.sumQH1C2 = Constraint(expr=40*(413-TC2E)+40*(TC2E-353)==2400)
modelNLP.sumQH2C1 = Constraint(expr=15*(423-TH2E)+15*(TH2E-303)==1800)

modelNLP.heater = Constraint(expr= 20*(408-TC1F)==200)
modelNLP.cooler = Constraint(expr= 30*(TH1F-333)==600)

modelNLP.EQQH1C1_1 = Constraint(expr=fC1E_1*(TC1E_1-TC1E)==QH1C1_1)
modelNLP.EQQH1C1_2 = Constraint(expr=fC1I_1*(TC1I_1-293)==QH1C1_2)
modelNLP.EQQH1C2_1 = Constraint(expr=40*(413-TC2E)==QH1C2_1)
modelNLP.EQQH1C2_2 = Constraint(expr=40*(TC2E-353)==QH1C2_2)
modelNLP.EQQH2C1_1 = Constraint(expr=15*(423-TH2E)==QH2C1_1)
modelNLP.EQQH2C1_2 = Constraint(expr=15*(TH2E-303)==QH2C1_2)

modelNLP.DT1H1C11 = Constraint(expr=443-TC1E_1>=10)
modelNLP.DT1H1C12 = Constraint(expr=TH1I_1-TC1E>=10)
modelNLP.DT2H1C11 = Constraint(expr=TH1E-TC1I_1>=10)
modelNLP.DT2H1C12 = Constraint(expr=TH1F-293>=10)
#modelNLP.DT1H1C21 = Constraint(expr=443-413>=10)
modelNLP.DT1H1C22 = Constraint(expr=TH1I_2-TC2E>=10)
modelNLP.DT2H1C21 = Constraint(expr=TH1E-TC2E>=10)
modelNLP.DT2H1C22 = Constraint(expr=TH1E_2-353>=10)
modelNLP.DT1H2C11 = Constraint(expr=423-TC1E_2>=10)
modelNLP.DT1H2C12 = Constraint(expr=TH2E-TC1E>=10)
modelNLP.DT2H2C11 = Constraint(expr=TH2E-TC1I_2>=10)
#modelNLP.DT2H2C12= Constraint(expr=303-293>=10)


modelNLP.DELT1H1C11 = Constraint(expr=443-TC1E_1==DT1H1C1_1)
modelNLP.DELT1H1C12 = Constraint(expr=TH1I_1-TC1E==DT1H1C1_2)
modelNLP.DELT2H1C11 = Constraint(expr=TH1E-TC1I_1==DT2H1C1_1)
modelNLP.DELT2H1C12 = Constraint(expr=TH1F-293==DT2H1C1_2)
modelNLP.DELT1H1C21 = Constraint(expr=443-413==DT1H1C2_1)
modelNLP.DELT1H1C22 = Constraint(expr=TH1I_2-TC2E==DT1H1C2_2)
modelNLP.DELT2H1C21 = Constraint(expr=TH1E-TC2E==DT2H1C2_1)
modelNLP.DELT2H1C22 = Constraint(expr=TH1E_2-353==DT2H1C2_2)
modelNLP.DELT1H2C11 = Constraint(expr=423-TC1E_2==DT1H2C1_1)
modelNLP.DELT1H2C12 = Constraint(expr=TH2E-TC1E==DT1H2C1_2)
modelNLP.DELT2H2C11 = Constraint(expr=TH2E-TC1I_2==DT2H2C1_1)
modelNLP.DELT2H2C12 = Constraint(expr=303-293==DT2H2C1_2)

modelNLP.EQ_LMTD11_1 = Constraint(expr=(2/3*(DT1H1C1_1*DT1H1C1_2)**0.5+1/6*(DT1H1C1_1+DT1H1C1_2))==LMTD11_1)
modelNLP.EQ_LMTD11_2 = Constraint(expr=(2/3*(DT2H1C1_1*DT2H1C1_2)**0.5+1/6*(DT2H1C1_1+DT2H1C1_2))==LMTD11_2)
modelNLP.EQ_LMTD12_1 = Constraint(expr=(2/3*(DT1H1C2_1*DT1H1C2_2)**0.5+1/6*(DT1H1C2_1+DT1H1C2_2))==LMTD12_1)
modelNLP.EQ_LMTD12_2 = Constraint(expr=(2/3*(DT2H1C2_1*DT2H1C2_2)**0.5+1/6*(DT2H1C2_1+DT2H1C2_2))==LMTD12_2)
modelNLP.EQ_LMTD21_1 = Constraint(expr=(2/3*(DT1H2C1_1*DT1H2C1_2)**0.5+1/6*(DT1H2C1_1+DT1H2C1_2))==LMTD21_1)
modelNLP.EQ_LMTD21_2 = Constraint(expr=(2/3*(DT2H2C1_1*DT2H2C1_2)**0.5+1/6*(DT2H2C1_1+DT2H2C1_2))==LMTD21_2)



In [236]:
modelNLP.Coste = Objective(expr=1200*(200/(1200*(10)))**0.6+1000*(QH1C1_1/(800*(LMTD11_1)))**0.6+1000*(QH1C1_2/(800*(LMTD11_2)))**0.6+1000*(QH1C2_1/(800*(LMTD12_1)))**0.6+1000*(QH1C2_2/(800*(LMTD12_2)))**0.6+1000*(QH2C1_1/(800*(LMTD21_1)))**0.6+1000*(QH2C1_2/(800*(LMTD21_2)))**0.6+1000*(600/(800*(10)))**0.6)

In [237]:
solver = 'ipopt'
resultsNLP = SolverFactory(solver).solve(modelNLP)
modelNLP.pprint()
resultsNLP.write()
Cost = value(modelNLP.Coste)
print('El coste usando {0} es de {1:.2f}'.format(solver,Cost))

46 Var Declarations
    DT1H1C1_1 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :    10 : 150.00000131725665 :   500 : False : False :  Reals
    DT1H1C1_2 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :    10 : 75.23359881935737 :   500 : False : False :  Reals
    DT1H1C2_1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :    10 :  30.0 :   500 : False : False :  Reals
    DT1H1C2_2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :    10 :  10.0 :   500 : False : False :  Reals
    DT1H2C1_1 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :    10 : 25.00000005250143 :   500 : False : False :  Reals
    DT1H2C1_2 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
 

In [238]:
# solver = 'gams'
# resultsNLP = SolverFactory(solver).solve(modelNLP, solver='CONOPT4', keepfiles=True, tee=True)
# modelNLP.pprint()
# resultsNLP.write()
# Cost = value(modelNLP.Coste)
# print('El coste usando {0} es de {1:.2f}'.format(solver,Cost))